# TP 01 - Fondamentaux NLP pour les Transformers

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Mieux comprendre comment une machine peut interpréter du texte

---

## Comment faire interpréter du texte par une machine ?

Si vous avez déjà travaillé avec des **images** le problème est de prime abord plus simple : une image peut naturellement se découper en une grille de pixels, chaque pixel est un nombre (0-255). Le réseau peut directement les traiter.

Mais pour le **texte** suivant ?

```
"L'apprentissage automatique révolutionne l'intelligence artificielle"
```

Ce n'est qu'une suite de caractères. Un réseau de neurones, par exemple, ne comprend que des **nombres**. Comment passer de l'un à l'autre ?

---

## Les problèmes à résoudre

Pour transformer du texte en représentation numérique exploitable, il faut résoudre **deux problèmes distincts** :

### Problème 1 : La tokenization

**Comment découper le texte suivant en morceaux ?**

```
"L'apprentissage automatique" → ???
```

Plusieurs stratégies sont possibles :
- Par mots : `["L'apprentissage", "automatique"]`
- Par caractères : `["L", "'", "a", "p", "p", "r", ...]`
- Par sous-mots : `["L'", "apprent", "issage", "auto", "matique"]`

Chaque stratégie a ses avantages et inconvénients. Nous les explorerons dans ce TP.

### Problème 2 : L'embedding

**Comment transformer ces morceaux en vecteurs qui ont du SENS ?**

Une fois le texte découpé, on pourrait simplement numéroter les tokens :
```
"chat" → 42
"voiture" → 46
"chien" → 73
```

Mais ces nombres sont **arbitraires**. Ils ne capturent pas que "chat" et "chien" sont des concepts proches (animaux domestiques), alors que "voiture" est complètement différent.

**Il faut trouver un moyen** de transformer chaque token en un **vecteur de plusieurs dimensions** où la **proximité géométrique** reflète la **proximité sémantique** :

```
"chat"    → [0.2, -0.5, 0.8, ...]   ┐
                                    ├─ vecteurs proches !
"chien"   → [0.3, -0.4, 0.7, ...]   ┘

"voiture" → [-0.8, 0.2, -0.3, ...]  ← vecteur éloigné
```

Plusieurs approches existent pour construire ces vecteurs. Dans ce TP, nous explorerons **Word2Vec**, une méthode remarquable qui a révolutionné le NLP en 2013.

---

## Plan du TP

| Section | Thème | Ce que vous apprendrez |
|---------|-------|------------------------|
| §2 | Tokenization | Les 3 stratégies (mots, caractères, BPE) |
| §3-4 | Embeddings | Comment les vecteurs capturent le sens (Word2Vec) |
| §5 | Attention (teaser) | Aperçu du mécanisme clé des Transformers |
| §6 | Récapitulatif | Synthèse des concepts |
| §7 | Ressources | Pour aller plus loin |
| §8 | Mini-projet | Pipeline NLP complet (optionnel) |

Commençons par le premier problème : **comment découper le texte ?**

## 0. Installation et imports

Exécutez cette cellule pour installer les dépendances nécessaires.

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy gensim -q

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

# Configuration
torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

---

# Partie 1 : La Tokenization

---

---

## 2. Tokenization : Découper le texte

La **tokenization** consiste à découper le texte en unités (tokens). Il existe plusieurs stratégies.

### 2.1 Tokenization par mots (Word-level)

La plus intuitive : on découpe sur les espaces et la ponctuation.

### Exercice 0a : Implémenter la tokenization par mots

Complétez la fonction `tokenize_words` ci-dessous. Elle doit :
- Séparer le texte sur les espaces
- Garder la ponctuation comme tokens séparés

**Indice** : Utilisez `re.findall()` avec le pattern `r"\w+|[^\w\s]"` qui capture les mots OU la ponctuation.

In [ ]:
# Tokenization simple par mots
def tokenize_words(text):
    """Tokenization basique par espaces et ponctuation."""
    import re
    # Sépare sur espaces et garde la ponctuation comme tokens
    return "" #TODO

texte = "Le chat mange la souris. La souris court vite !"
tokens = tokenize_words(texte)

print(f"Texte : {texte}")
print(f"Tokens : {tokens}")
print(f"Nombre de tokens : {len(tokens)}")

**Problème** : Le vocabulaire peut devenir énorme !

Ajoutez les noms propres, les néologismes, les mots étrangers, les fautes de frappe... Le vocabulaire explose.

À un moment, il faut **fixer une taille de vocabulaire** (ex: 50 000 mots). Mais alors, que faire des mots inconnus ?

Imaginons un vocabulaire constitué au préalable :
```
["le", "chat", "mange", ...] (50 000 mots)
```
Si un mot traité n'appartient pas au vocabulaire, l'information est perdue
```
Nouveau mot : "transformers" → <UNK> ?
```
→ Pas idéal.

### 2.2 Tokenization par caractères (Character-level)

Une solution : découper caractère par caractère. Plus de mots inconnus !

### Exercice 0b : Implémenter la tokenization par caractères

Complétez la fonction `tokenize_chars` ci-dessous.

**Indice** : En Python, une chaîne est déjà itérable caractère par caractère...

In [ ]:
def tokenize_chars(text):
    """Tokenization par caractères."""
    return "" #TODO

texte = "Le chat dort."
tokens = tokenize_chars(texte)

print(f"Texte : {texte}")
print(f"Tokens : {tokens}")
print(f"Nombre de tokens : {len(tokens)}")

**Problème historique** : Séquences très longues ! "anticonstitutionnellement" = 25 tokens.

Le modèle doit "réapprendre" que `c-h-a-t` forme le concept de chat. Le coût computationnel était longtemps considéré comme prohibitif.

---

> 📚 **Des travaux récents montrent que la tokenization byte-level peut devenir compétitive**
>
> 
> **Bolmo** (Allen AI, 2025) opère directement sur les **bytes UTF-8** (256 tokens possibles) avec une architecture adaptée :
> - Un encodeur local (mLSTM) traite les bytes
> - Un "boundary predictor" regroupe les bytes en patches de taille variable
> - Le Transformer traite ces patches (pas les bytes bruts)
> 
> **Avantages** : pas de vocabulaire figé, robuste aux typos, meilleure compréhension caractère.

Pour ce TP, nous utiliserons **BPE** que nous allons voir tout de suite et qui reste le standard actuel, mais gardez en tête que le domaine évolue rapidement !

### 2.3 Tokenization Subword : BPE (Byte Pair Encoding)

Les deux approches précédentes ont des défauts :
- **Mots** : vocabulaire énorme + mots inconnus
- **Caractères** : séquences trop longues + perte de sens

**BPE** (Byte Pair Encoding) est un **compromis intelligent** utilisé par GPT, BERT, et une grande partie des LLMs modernes.

---

#### Le principe

BPE construit son vocabulaire en analysant un grand corpus de texte :

1. **Départ** : vocabulaire = tous les caractères
2. **Répéter** : trouver la paire de tokens adjacents la plus fréquente → la fusionner en un nouveau token
3. **Stop** : quand le vocabulaire atteint la taille voulue (ex: 50 000 tokens)

---

#### L'algorithme pas à pas

Pour comprendre, prenons un **corpus artificiel simplifié** :
```
"smartphone smartphone smartphone smartwatch smartwatch phone phone"
```

**Étape 0 : Partir des caractères**
```
Vocabulaire : {s, m, a, r, t, p, h, o, n, e, w, c}
```

**Étapes suivantes : Fusionner les paires les plus fréquentes**

| Étape | Paire la + fréquente | Nouveau token |
|-------|---------------------|---------------|
| 1 | (s, m) | "sm" |
| 2 | (sm, a) | "sma" |
| 3 | (sma, r) | "smar" |
| 4 | (smar, t) | "smart" |
| 5 | (p, h) | "ph" |
| 6 | (ph, o) | "pho" |
| 7 | (pho, n) | "phon" |
| 8 | (phon, e) | "phone" |

**Résultat :**
```
"smartphone" → ["smart", "phone"]  ← 2 tokens réutilisables !
"smartwatch" → ["smart", "watch"]
"phone"      → ["phone"]
```

---

#### Pourquoi c'est malin ?

Un mot **jamais vu** comme "smartcar" sera découpé en :
```
"smartcar" → ["smart", "car"]
```

Le modèle connaît déjà "smart" ! Pas besoin de token `<UNK>`.

**Bonus** : les sous-mots fréquents ont de **bons embeddings** (on verra pourquoi dans la section Word2Vec). Donc même un mot rare peut bénéficier de représentations de qualité via ses composants.

**Nous obtenons le meilleur des deux mondes** :
- Mots fréquents → tokens entiers (efficace)
- Mots rares/nouveaux → sous-mots connus (robuste)

Bien que ce modèle soit très performant dans un grand nombre de cas, il faut rester conscient que certains mots très spécifiques peuvent avoir une représentation de moindre qualité.

**Ressource** : [Explication détaillée des tokenizers (FR)](https://lbourdois.github.io/blog/nlp/Les-tokenizers/)

---

#### En pratique : le tokenizer de GPT-2

Voyons maintenant comment un **vrai tokenizer BPE** découpe du texte. Nous allons utiliser celui de GPT-2 via la bibliothèque Hugging Face `transformers`.

In [ ]:
# Démonstration avec un vrai tokenizer (GPT-2)
!pip install transformers -q

from transformers import GPT2Tokenizer

gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

textes = [
    "Le chat mange.",
    "anticonstitutionnellement",
    "Hello world!",
    "Transformers are amazing!"
]

print("=== Tokenization BPE (GPT-2) ===")
for texte in textes:
    tokens = gpt2_tokenizer.tokenize(texte)
    ids = gpt2_tokenizer.encode(texte)
    print(f"\n'{texte}'")
    print(f"  Tokens : {tokens}")
    print(f"  IDs    : {ids}")

**Observations** :
- Les mots courants anglais sont souvent des tokens uniques
- Les mots français/rares sont découpés
- Le caractère `Ġ` indique un espace avant le token

### Exercice 1 : Comparer les tokenizations

In [ ]:
# ============================================
# EXERCICE 1 : Comparer les tokenizations
# ============================================

texte_test = "L'intelligence artificielle révolutionne le monde."

# TODO: Tokenizer avec les 3 méthodes et comparer le nombre de tokens

# 1. Par mots
tokens_mots = None

# 2. Par caractères  
tokens_chars = None

# 3. BPE (GPT-2)
tokens_bpe = None

print(f"Texte : {texte_test}")
print(f"\nMots      : {len(tokens_mots) if tokens_mots else '?'} tokens")
print(f"Caractères: {len(tokens_chars) if tokens_chars else '?'} tokens")
print(f"BPE       : {len(tokens_bpe) if tokens_bpe else '?'} tokens")

### 2.4 Construction d'un vocabulaire : Exemple avec le word tokenizer

Une fois la stratégie choisie, on construit un **vocabulaire** : une table de correspondance token ↔ index.

In [ ]:
# Construire un vocabulaire simple
corpus = [
    "le chat mange",
    "le chien dort",
    "la souris court"
]

# Collecter tous les tokens uniques
all_tokens = set()
for phrase in corpus:
    all_tokens.update(tokenize_words(phrase))

# Créer le vocabulaire avec tokens spéciaux
vocab = {"<PAD>": 0, "<UNK>": 1}  # Padding et Unknown
for i, token in enumerate(sorted(all_tokens)):
    vocab[token] = i + 2

# Vocabulaire inverse
id_to_token = {v: k for k, v in vocab.items()}

print("Vocabulaire :")
for token, idx in vocab.items():
    print(f"  {idx}: '{token}'")

# Encoder une phrase
def tokens_to_ids(text, vocab):
    return [vocab.get(t, vocab["<UNK>"]) for t in tokenize_words(text)]

def ids_to_token(ids, id_to_token):
    return [id_to_token[i] for i in ids]

phrase = "le chat court"
ids = tokens_to_ids(phrase, vocab)
tokens = ids_to_token(ids, id_to_token)

print(f"\nPhrase : '{phrase}'")
print(f"Conversion des tokens en ids : {ids}")
print(f"Conversion des ids en token : {tokens}")

In [ ]:
# ============================================
# EXERCICE 1b : Manipuler le vocabulaire
# ============================================

# Notre vocabulaire actuel :
print("Vocabulaire actuel :", list(vocab.keys()))

# Partie A : Ajouter des tokens spéciaux
# TODO: Ajoutez les tokens "<START>" et "<END>" au vocabulaire
# Indice : vocab["<START>"] = len(vocab)


# Partie B : Encoder une phrase inconnue
# TODO: Encodez la phrase "le hamster mange" avec tokens_to_ids()
# Quel token devient <UNK> ? Pourquoi ?


# Partie C : Étendre le vocabulaire
# TODO: Ajoutez "hamster" au vocabulaire, puis ré-encodez la phrase
# Comparez les résultats

---

## 3. Embeddings : Des indices aux vecteurs

### Le problème des indices

Les indices (0, 1, 2, ...) n'ont pas de **sens sémantique**. 

- `chat = 3` et `chien = 5` → sont-ils proches ? (oui, ce sont des animaux et ils ont beaucoup de choses en commun)
- `chat = 3` et `voiture = 4` → sont-ils proches ? (non, beaucoup moins que chien et chat)

Ces indices ne permettent pas de mesurer cette proximité !

### Approches classiques pour encoder des phrases : Bag-of-Words et TF-IDF

Historiquement, on représentait un texte par un vecteur de la taille du vocabulaire :
- **Bag-of-Words** : compter les occurrences de chaque mot
- **TF-IDF** : pondérer par la rareté des mots dans le corpus

```
Exemple simplifié (BOW vs TF-IDF) :

Corpus : ["le chat mange", "le chat dort"]
Vocabulaire : [le, chat, mange, dort]

BOW (comptage brut) :
  "le chat mange" → [1, 1, 1, 0]
  "le chat dort"  → [1, 1, 0, 1]

TF-IDF (pondéré par rareté) :
  "le chat mange" → [0.3, 0.3, 0.7, 0]    ← "mange" pèse plus (mot distinctif)
  "le chat dort"  → [0.3, 0.3, 0, 0.7]    ← "dort" pèse plus (mot distinctif)
                     ↑    ↑
              mots communs → poids réduit
```

En pratique, le vocabulaire contient 50 000+ mots → vecteurs **sparse** (majoritairement des zéros) :

```
"le chat dort" → [0, 0, ..., 1, ..., 0, 1, ..., 0]  (50 000 dimensions)
                              ↑         ↑
                            chat      dort
```

Pour TF-IDF et BOW, on obtient des vecteurs qui permettent de comparer des morceaux de texte entre eux (descriptions de produits sur un site marchand par exemple). Pour autant, ils ne permettent pas de capturer le sens des mots et leurs proximités relatives. Une méthode apparue en 2013 (Word2Vec) a permis d'apporter cette compréhension plus profonde, de manière automatisée et sans supervision.

---

## 4. Word2Vec : Apprendre des embeddings qui ont du sens

Comme son nom l'indique, Word2Vec transforme des mots en vecteurs. Mais comment apprend-il des vecteurs où "chat" et "chien" sont vraiment proches ?

---

### L'intuition fondamentale

> **"Tu connais un mot par les mots qui l'entourent"** (hypothèse distributionnelle)

Observez ces phrases :
```
"Le chat mange sa pâtée"
"Le chat dort sur le canapé"  
"Mon chat joue avec une balle"

"Le chien mange sa pâtée"
"Le chien dort sur le canapé"
"Mon chien joue avec une balle"
```

"Chat" et "chien" apparaissent dans les **mêmes contextes**. Word2Vec va leur attribuer des vecteurs similaires.

---

### L'architecture de Word2Vec

Word2Vec repose sur une architecture similaire à un réseau de neurones **étonnamment simple** : une entrée, une couche cachée, une sortie. Pas d'activation (simple multiplication matricielle).

#### Le réseau

```
      ENTRÉE                 COUCHE CACHÉE              SORTIE
     (one-hot)               (embeddings)              (softmax)
  
   ┌───┐                                              ┌───────┐
   │ 0 │  "le"                                        │ 0.02  │ "le"
   ├───┤                                              ├───────┤
   │ 0 │  "chat"             ┌───────────┐            │ 0.41  │ "chat"
   ├───┤                     │           │            ├───────┤
   │ 1 │  "noir"  ────────▶  │  vecteur  │  ────────▶ │ 0.05  │ "noir"
   ├───┤            W        │  128 dim  │     W'     ├───────┤
   │ 0 │  "dort"             │           │            │ 0.38  │ "dort"
   ├───┤                     └───────────┘            ├───────┤
   │ 0 │  "sur"                                       │ 0.14  │ "sur"
   └───┘                                              └───────┘
  
  V dimensions              D dimensions              V dimensions
  (taille vocab)            (ex: 128)                 (probabilités)
```

| Couche | Dimensions | Ce qu'elle contient |
|--------|------------|---------------------|
| Entrée | V (ex: 50 000) | Vecteur one-hot du mot |
| Cachée | D (ex: 128) | **Le vecteur qu'on veut récupérer** |
| Sortie | V (ex: 50 000) | Probabilité de chaque mot |

---

#### Le but : récupérer la couche cachée

L'objectif de Word2Vec n'est **pas** de faire des prédictions. C'est de construire de bons vecteurs.

La **matrice W** (entre l'entrée et la couche cachée) contient tous les embeddings :

```
Matrice W (V × D)
                    D dimensions
              ┌─────────────────────┐
   "le"     → │ 0.2  -0.1  0.5 ... │
   "chat"   → │ 0.3  -0.4  0.7 ... │ ← Ces deux lignes
   "chien"  → │ 0.3  -0.3  0.6 ... │ ← sont proches !
   "noir"   → │ 0.1   0.5 -0.2 ... │
   ...        │        ...         │
              └─────────────────────┘
```

**Après l'entraînement** :
- On **garde** la matrice W → c'est notre table d'embeddings
- On **jette** tout le reste (matrice W', couche de sortie)

La sortie n'était qu'un **prétexte** pour entraîner le réseau !

---

#### L'entraînement : deux techniques

Pour que les vecteurs capturent le sens des mots, on entraîne le réseau à prédire les relations entre mots voisins dans un corpus.

**Deux approches symétriques existent :**

##### Skip-gram : mot central → mots voisins

On donne un mot, le réseau prédit les mots qui l'entourent.

```
Phrase : "Le chat noir dort sur"
                  ↑
             mot central

Fenêtre de contexte (±1 mot) :
    Entrée  : "noir"
    Cibles  : "chat", "dort" (traités un par un)
```

```
    "noir"                     "chat" ?
       │                          ↑
       ▼                          │
   ┌───────┐    vecteur     ┌─────────┐
   │   W   │ ─────────────▶ │   W'    │
   └───────┘   (128 dim)    └─────────┘
```

##### CBOW : mots voisins → mot central

On donne les mots du contexte, le réseau prédit le mot du milieu.

```
Phrase : "Le chat noir dort sur"
              ↑         ↑
            contexte (±1)

Entrée  : "chat" + "dort" (moyennés)
Cible   : "noir"
```

```
"chat" + "dort"                "noir" ?
       │                          ↑
       ▼                          │
   ┌───────┐    vecteur     ┌─────────┐
   │   W   │ ─────────────▶ │   W'    │
   └───────┘   (128 dim)    └─────────┘
```

##### Comparaison

| | Skip-gram | CBOW |
|--|-----------|------|
| Entrée | 1 mot | plusieurs mots |
| Sortie | prédire les voisins | prédire le mot central |
| Mots rares | ✅ meilleur | moins bon |
| Vitesse | plus lent | ✅ plus rapide |

En pratique, **Skip-gram** est plus utilisé car il donne de meilleurs résultats sur les mots peu fréquents.

---

#### Conséquence : mots fréquents vs mots rares

Plus un mot est **fréquent** dans le corpus, plus son embedding est ajusté souvent → meilleure qualité.

Un mot vu 100 000 fois aura un excellent embedding. Un mot vu 3 fois restera proche de son initialisation aléatoire.

> C'est pour ça que BPE aide : un mot rare comme "smartcar" est découpé en "smart" + "car", deux sous-mots très fréquents avec d'excellents embeddings !

---

#### Limitation : l'ordre des mots est ignoré

Word2Vec traite chaque paire (mot, voisin) **indépendamment**. Il ne sait pas quel mot vient avant ou après.

```
"Le chat mange la souris"
"La souris mange le chat"

→ Mêmes paires d'entraînement !
→ Word2Vec ne voit pas la différence
```

C'est le **mécanisme d'attention combiné au positional encoding** (section 5) qui permettra de capturer l'ordre et les relations entre positions.

---

> 💡 **En pratique** : Les implémentations réelles (gensim, FastText...) ajoutent des optimisations pour accélérer l'entraînement sur de gros vocabulaires. L'architecture de base reste la même.

> 📚 **Pour aller plus loin** : [The Illustrated Word2Vec](https://jalammar.github.io/illustrated-word2vec/) — Visualisations détaillées de l'architecture et de l'entraînement.

---

### Pourquoi les analogies marchent ?

Après entraînement, les vecteurs encodent des **relations** :

```
vecteur("roi") - vecteur("homme") ≈ vecteur("reine") - vecteur("femme")
```

Autrement dit, la "direction" homme→femme dans l'espace vectoriel est la même que roi→reine :

```
        homme ─────────────→ femme
          ↑    (même         ↑
          │   direction)     │
         roi ─────────────→ reine
```

C'est pour ça que `roi - homme + femme ≈ reine` fonctionne !

---

### Mesurer la similarité : Distance cosinus

Avec des vecteurs, on peut mesurer la **similarité** entre mots :

$$\text{similarité}(A, B) = \cos(\theta) = \frac{A \cdot B}{\|A\| \|B\|}$$

- Résultat entre -1 (opposés) et 1 (identiques)
- 0 = orthogonaux (pas de relation)

---

### Chargeons un modèle pré-entraîné

Nous allons utiliser **GloVe** (similaire à Word2Vec), entraîné sur Wikipedia.

In [ ]:
# Charger un modèle pré-entraîné (GloVe, similaire à Word2Vec)
import gensim.downloader as api

print("Chargement du modèle GloVe (peut prendre 1-2 min)...")
model = api.load("glove-wiki-gigaword-100")  # 100 dimensions, entraîné sur Wikipedia
print(f"Modèle chargé ! Vocabulaire : {len(model)} mots")

In [ ]:
# Explorer les similarités
print("=== Mots similaires à 'king' ===")
for word, score in model.most_similar("king", topn=5):
    print(f"  {word}: {score:.4f}")

print("\n=== Mots similaires à 'computer' ===")
for word, score in model.most_similar("computer", topn=5):
    print(f"  {word}: {score:.4f}")

In [ ]:
# ============================================
# EXERCICE 2 : Explorer les similarités
# ============================================

# Partie A : Trouver les mots similaires à "france", "cat", "happy"
# TODO: Utilisez model.most_similar(mot, topn=5)


# Partie B : Exploration libre
# TODO: Trouvez une paire de mots avec similarité > 0.7
# Indice : model.similarity("mot1", "mot2") retourne un score entre -1 et 1


# Partie C : Trouver l'intrus
# TODO: Utilisez model.doesnt_match(["mot1", "mot2", "mot3", "mot4"])
# Exemple : model.doesnt_match(["breakfast", "lunch", "dinner", "car"])
# Testez avec vos propres listes !

In [ ]:
# La magie des analogies : king - man + woman = ?
print("=== Analogie : king - man + woman = ? ===")
result = model.most_similar(positive=["king", "woman"], negative=["man"], topn=3)
for word, score in result:
    print(f"  {word}: {score:.4f}")

print("\n=== Analogie : paris - france + italy = ? ===")
result = model.most_similar(positive=["paris", "italy"], negative=["france"], topn=3)
for word, score in result:
    print(f"  {word}: {score:.4f}")

In [ ]:
# ============================================
# EXERCICE 3 : Maîtriser les analogies
# ============================================

# Partie A : Tester ces analogies classiques
# - "berlin" - "germany" + "france" = ?
# - "good" - "better" + "bad" = ?
# - "walked" - "walk" + "swim" = ?

# TODO: Syntaxe : model.most_similar(positive=["A", "C"], negative=["B"], topn=3)


# Partie B : Inventer une analogie qui fonctionne
# TODO: Trouvez une analogie originale qui donne le résultat attendu
# Exemples de domaines : métiers, pays/capitales, animaux, verbes...


# Partie C : Trouver une analogie qui échoue
# TODO: Trouvez une analogie qui devrait marcher logiquement mais échoue
# Expliquez pourquoi dans un commentaire (indice : fréquence des mots, biais du corpus...)

In [ ]:
# Visualisation des embeddings en 2D
!pip install scikit-learn -q
from sklearn.decomposition import PCA

# Sélectionner quelques mots
words = ["king", "queen", "man", "woman", "prince", "princess",
         "cat", "dog", "lion", "tiger",
         "car", "bus", "train", "plane"]

# Récupérer leurs vecteurs
vectors = np.array([model[w] for w in words])

# Réduire à 2D avec PCA
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(vectors)

# Visualiser
plt.figure(figsize=(12, 8))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], c='blue', s=100)

for i, word in enumerate(words):
    plt.annotate(word, (vectors_2d[i, 0] + 0.1, vectors_2d[i, 1] + 0.1), fontsize=12)

plt.title("Embeddings GloVe projetés en 2D")
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.grid(True, alpha=0.3)
plt.show()

print("Observation : Les mots de même catégorie sont regroupés !")

In [ ]:
# ============================================
# EXERCICE 4 : Visualisation personnalisée
# ============================================

# TODO: Créez votre propre visualisation avec 15-20 mots de votre choix
# Choisissez des mots de 3-4 catégories différentes (ex: sports, émotions, pays, métiers)

# Vos mots (modifiez cette liste) :
my_words = [
    # Catégorie 1 (ex: sports) : 
    
    # Catégorie 2 (ex: émotions) : 
    
    # Catégorie 3 (ex: pays) : 
    
    # Catégorie 4 (ex: métiers) : 
]

# TODO: Vérifiez que tous vos mots sont dans le vocabulaire
# for word in my_words:
#     if word not in model:
#         print(f"'{word}' n'est pas dans le vocabulaire !")

# TODO: Copiez et adaptez le code de visualisation de la cellule précédente
# Remplacez 'words' par 'my_words'


# QUESTION : Les mots de même catégorie sont-ils regroupés ? 
# Y a-t-il des surprises ?

---

## 5. Teaser : Le mécanisme d'attention

Maintenant que nous savons représenter du texte (tokenization + embeddings), la prochaine étape est de permettre aux mots de **communiquer entre eux**.

C'est le rôle du **mécanisme d'attention**, que nous verrons en détail au prochain TP.

### L'idée

> Pour comprendre un mot, il faut regarder les autres mots de la phrase.

Exemple : *"Le chat qui dormait sur le canapé a sauté"*
- Pour comprendre **"a sauté"** → regarder **"chat"** (le sujet)
- Pour comprendre **"dormait"** → regarder **"chat"** et **"canapé"**

### Visualisation

In [ ]:
# Matrice d'attention simulée
phrase = ["Le", "chat", "mange", "la", "souris"]

# Chaque ligne = un mot qui "regarde" les autres
# Valeurs = poids d'attention (somme = 1 par ligne)
attention = torch.tensor([
    [0.8, 0.1, 0.05, 0.03, 0.02],  # "Le" regarde surtout lui-même
    [0.1, 0.7, 0.1, 0.05, 0.05],   # "chat" regarde surtout lui-même
    [0.05, 0.4, 0.4, 0.05, 0.1],   # "mange" regarde "chat" et lui-même
    [0.02, 0.03, 0.05, 0.8, 0.1],  # "la" regarde surtout lui-même
    [0.02, 0.1, 0.2, 0.08, 0.6],   # "souris" regarde "mange" et elle-même
])

# Visualisation
plt.figure(figsize=(8, 6))
plt.imshow(attention, cmap='Blues')
plt.xticks(range(5), phrase)
plt.yticks(range(5), phrase)
plt.xlabel("Mots regardés")
plt.ylabel("Mots qui regardent")
plt.title("Qui regarde qui ? (Matrice d'attention)")
plt.colorbar(label="Poids d'attention")

for i in range(5):
    for j in range(5):
        plt.text(j, i, f'{attention[i,j]:.2f}', 
                ha='center', va='center',
                color='white' if attention[i,j] > 0.5 else 'black')
plt.show()

print("Le verbe 'mange' regarde fortement 'chat' (son sujet) !")

**Ce qu'on voit** :
- Chaque mot peut "regarder" tous les autres mots
- Les poids indiquent l'importance de chaque relation
- Le modèle **apprend** ces poids pendant l'entraînement

**Au prochain TP**, nous verrons :
- Comment calculer cette matrice d'attention
- Les concepts Query, Key, Value
- L'architecture complète du Transformer

---

## 6. Récapitulatif

### Le pipeline NLP

| Étape | Entrée | Sortie | Rôle |
|-------|--------|--------|------|
| **Tokenization** | Texte brut | Liste de tokens | Découper le texte |
| **Vocabulaire** | Tokens | Indices | Table token ↔ ID |
| **Embedding** | Indices | Vecteurs denses | Sens sémantique |

### Points clés

1. **Tokenization BPE** : meilleur compromis entre mots et caractères
2. **Embeddings** : transforment les mots en vecteurs comparables
3. **Word2Vec** : montre que les embeddings capturent le sens (analogies !)
4. **Similarité cosinus** : mesure la proximité entre vecteurs

### Prochaine session

Nous verrons le **mécanisme d'attention** : comment les mots "communiquent" entre eux pour se comprendre mutuellement.

---

## 7. Pour aller plus loin (optionnel)

### Ressources

- [Les tokenizers en NLP (FR)](https://lbourdois.github.io/blog/nlp/Les-tokenizers/) - Excellent article en français
- [The Illustrated Word2Vec](https://jalammar.github.io/illustrated-word2vec/) - Visualisations très claires
- [Bolmo: Byte-level Language Models (Allen AI, 2025)](https://allenai.org/blog/bolmo) - Une alternative à BPE ?

### Expérimentations suggérées

1. Tester d'autres analogies Word2Vec
2. Visualiser les embeddings de votre choix
3. Comparer différents tokenizers (BERT, GPT-2, etc.)

---

## 8. Mini-projet de synthèse (optionnel)

Mettez en pratique tout ce que vous avez appris en analysant vos propres phrases. Ce projet combine tokenization, vocabulaire et embeddings.

In [ ]:
# ============================================
# MINI-PROJET : Pipeline NLP complet
# ============================================

# Choisissez 3 phrases en anglais sur un thème commun
mes_phrases = [
    "The quick brown fox jumps over the lazy dog",
    "A fast red cat leaps across the sleepy mouse", 
    "The slow black bear walks around the tired wolf"
]

# TODO Étape 1 : Tokenisez chaque phrase avec les 3 méthodes
# Comparez le nombre de tokens
# for phrase in mes_phrases:
#     print(f"Phrase: {phrase}")
#     print(f"  Mots: {len(tokenize_words(phrase))}")
#     print(f"  Chars: {len(tokenize_chars(phrase))}")
#     print(f"  BPE: {len(gpt2_tokenizer.tokenize(phrase))}")


# TODO Étape 2 : Pour la tokenization par mots, construisez un vocabulaire unifié
# Indice : collectez les tokens de toutes les phrases dans un set


# TODO Étape 3 : Récupérez les embeddings GloVe des mots communs aux 3 phrases
# Indice : utilisez des sets et l'intersection (&)


# TODO Étape 4 : Calculez la similarité moyenne entre les phrases
# Indice : moyennez les embeddings de chaque phrase, puis calculez la similarité cosinus
# from numpy.linalg import norm
# def cosine_sim(v1, v2): return np.dot(v1, v2) / (norm(v1) * norm(v2))


# TODO Étape 5 : Visualisez les mots des 3 phrases sur un même graphique
# Utilisez des couleurs différentes pour chaque phrase


# QUESTIONS DE RÉFLEXION :
# 1. Quelle méthode de tokenization donne le vocabulaire le plus compact ?
# 2. Les phrases similaires ont-elles des embeddings moyens proches ?
# 3. Quelles limites de Word2Vec observez-vous sur vos phrases ?